In [1]:
from utils import *
import datasets
import os
import keras

[nltk_data] Downloading package stopwords to /home/jose/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


/home/jose/scratch/venv/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Powered by https://blog.keras.io/a-ten-minute-introduction-to-sequence-to-sequence-learning-in-keras.html
    https://stackoverflow.com/questions/47923370/keras-bidirectional-lstm-seq2seq

In [ ]:
#!./download_cornell.sh

In [3]:
data = datasets.readCornellData("data/cornell", max_len=20)

100%|██████████| 83097/83097 [00:03<00:00, 23531.72it/s]


In [4]:
#! ./download_opensubs.sh

In [5]:
#data = datasets.readOpensubsData("data/opensubs/", max_len=20)

In [6]:
# Tokenization
from collections import Counter
from itertools import chain
#start_symbol = '^'
end_symbol = '$'
padding_symbol = '#'
unk_symbol = "|"
start_symbol="^"
all_sym = [start_symbol,end_symbol, padding_symbol, unk_symbol] + \
    list(Counter(chain.from_iterable(chain.from_iterable(data))).keys())

In [7]:
char2id = {symbol:i for i, symbol in enumerate(all_sym)}
id2char = {i:symbol for symbol, i in char2id.items()}

In [8]:
def w2i(sentence, word2id=char2id, padded_len=22):
    """ Converts a sequence of words to a padded sequence of their ids.

      sentence: a string, input/output sequence of symbols.
      word2id: a dict, a mapping from original symbols to ids.
      padded_len: an integer, a desirable length of the sequence.

      result: a tuple of (a list of ids, an actual length of sentence).
    """
    # Account for start and end
    sentence="^"+sentence
    sentence = [i for i in sentence]

    
  

    sent_ids = [word2id[i] if i in word2id.keys() else word2id["|"]
                for i in sentence]
    
    sent_len = len(sent_ids[:padded_len-1])+1
    sent_ids = sent_ids[:padded_len-1]+[word2id["$"]] + \
        [word2id["#"]]*(padded_len-len(sent_ids)-1)

   # return (sent_ids, sent_len-2)
    return sent_ids

In [9]:
def ids_to_sentence(ids, id2word):
    """ Converts a sequence of ids to a sequence of symbols.
    
          ids: a list, indices for the padded sequence.
          id2word:  a dict, a mapping from ids to original symbols.

          result: a list of symbols.
    """
 
    return [id2word[i] for i in ids] 

In [10]:
X = np.array([w2i(i[0]) for i in data])
y = np.array([w2i(i[1]) for i in data])

In [11]:
more_pad=np.ones((y.shape[0],1),dtype=np.int)*char2id["#"]

In [12]:
y_shift=np.concatenate([y[:,1:],more_pad],axis=1)

In [13]:
#Onehot encode
X_ohot = np.zeros(shape=(X.shape[0],X.shape[1],len(id2char)),dtype="float32")
y_ohot = np.zeros(shape=(X.shape[0],X.shape[1],len(id2char)),dtype="float32")
y_shift_ohot = np.zeros(shape=(X.shape[0],X.shape[1],len(id2char)),dtype="float32")

In [14]:
for i in range(X.shape[0]):
    for k,character in enumerate(X[i,:]):
        X_ohot[i,k,character]=1
    for k,character in enumerate(y[i,:]):
        y_ohot[i,k,character]=1
    for k,character in enumerate(y_shift[i,:]):
        y_shift_ohot[i,k,character]=1

In [99]:
from keras import backend as K
K.clear_session()

In [141]:
from keras.layers import LSTM, Input, Dense, Embedding,Bidirectional,Concatenate
from keras.models import Sequential, Model

latent_dim = 150
num_tokens = len(id2char)
batch_size=64
epochs=12
max_len=X.shape[1]

# Encoder
encoder_input = Input(shape=(None,num_tokens))

#pre_encoder=LSTM(latent_dim, return_state=True,dropout=0.5,return_sequences=True)(encoder_input)
encoder=LSTM(latent_dim, return_state=True,return_sequences=True)


encoder_outputs, state_h, state_c=encoder(encoder_input)
#encoder_outputs, forward_h, forward_c, backward_h, backward_c = encoder(encoder_input)
encoder_states=[state_h, state_c]

# Decoder
decoder_input=Input(shape=(None,num_tokens))
#pre_decoder=LSTM(latent_dim, return_state=True,dropout=0.5,return_sequences=True)
decoder_lstm=LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs,_,_=decoder_lstm(
   decoder_input, initial_state=encoder_states)
decoder_dense=Dense(num_tokens, activation="softmax")
decoder_output=decoder_dense(decoder_outputs)

# Model
model=Model([encoder_input, decoder_input], decoder_output)

In [142]:
from keras.optimizers import Adam
adam =Adam(lr=0.005)

In [143]:
model.compile(loss="categorical_crossentropy",optimizer=adam,metrics=["accuracy"])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_15 (InputLayer)           (None, None, 41)     0                                            
__________________________________________________________________________________________________
input_16 (InputLayer)           (None, None, 41)     0                                            
__________________________________________________________________________________________________
lstm_13 (LSTM)                  [(None, None, 150),  115200      input_15[0][0]                   
__________________________________________________________________________________________________
lstm_14 (LSTM)                  [(None, None, 150),  115200      input_16[0][0]                   
                                                                 lstm_13[0][1]                    
          

In [149]:
K.set_value(model.optimizer.lr, 0.001)

In [150]:
model.fit(x=[X_ohot, y_ohot], y=y_shift_ohot,
          batch_size=batch_size,
          epochs=10,
          validation_split=0.2)

Train on 19833 samples, validate on 4959 samples
Epoch 1/10
19833/19833 [==============================] - 17s 873us/step - loss: 0.8278 - acc: 0.7406 - val_loss: 0.8708 - val_acc: 0.7321
Epoch 2/10
19833/19833 [==============================] - 17s 833us/step - loss: 0.8145 - acc: 0.7451 - val_loss: 0.8655 - val_acc: 0.7335
Epoch 3/10
19833/19833 [==============================] - 16s 824us/step - loss: 0.8051 - acc: 0.7482 - val_loss: 0.8614 - val_acc: 0.7349
Epoch 4/10
19833/19833 [==============================] - 17s 859us/step - loss: 0.7964 - acc: 0.7501 - val_loss: 0.8579 - val_acc: 0.7368
Epoch 5/10
19833/19833 [==============================] - 17s 882us/step - loss: 0.7878 - acc: 0.7528 - val_loss: 0.8557 - val_acc: 0.7362
Epoch 6/10
19833/19833 [==============================] - 19s 968us/step - loss: 0.7800 - acc: 0.7551 - val_loss: 0.8525 - val_acc: 0.7378
Epoch 7/10
19833/19833 [==============================] - 20s 991us/step - loss: 0.7716 - acc: 0.7579 - val_loss: 0.8

In [153]:
encoder_model = Model(encoder_input, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

decoder_output, state_hd, state_cd = decoder_lstm(
    decoder_input, initial_state=decoder_states_inputs)
decoder_states = [state_hd, state_cd]
decoder_output = decoder_dense(decoder_output)
decoder_model = Model(
    [decoder_input] + decoder_states_inputs,
    [decoder_output] + decoder_states)

In [92]:
def decode_sequence(input_seq, encoder_model, decoder_model, char2id, max_len):
    input_seq = ohot_input(input_seq, char2id, max_len)
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, len(char2id)))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, char2id["^"]] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
       
        sampled_char = id2char[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '$' or
                len(decoded_sentence) > max_len):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    decoded_sentence = (decoded_sentence.replace("$", "")
                        .replace("i m", "i'm")
                        .replace("what s", "what's")
                        .replace("don t", "don't")
                        .replace("that s", "that's")
                        .replace("you re", "you're")
                        .replace("he s", "he's")
                       )

    return decoded_sentence

In [108]:
def ohot_input(x,char2id,max_len):
    nums=w2i(x,word2id=char2id,padded_len=max_len)
    out = np.zeros((1,max_len,len(char2id)))
    
    for k,character in enumerate(nums):
        out[0,k,character]=1
    return out

In [156]:
decode_sequence("how are you",encoder_model,decoder_model,char2id,max_len)

'i dont know'